In [14]:
%load_ext autoreload
%autoreload

import networkx as nx
import networkx.algorithms as algos
from networkx.algorithms import approximation
from networkTrips import organizeTrips
from timeUtils import clock, elapsed, getDateTime
from haversine import haversine
from ioUtils import loadJoblib
from geocluster import geoClusters
from geoUtils import convertMetersToLat, convertLatToMeters, convertMetersToLong, convertLongToMeters
from geoclusterUtils import genCenters, genCluster, genClusters, genTripsBetweenClusters

In [30]:
class network():
    def __init__(self, directed=True, debug=False):
        self.debug = debug
        self.directed = directed
        
        self.orderedEdges    = None
        self.edgeDict        = None
        self.orderedVertices = None
        self.nodeDict        = None
        
        if self.directed is True:
            self.g = nx.DiGraph()
        else:
            self.g = nx.Graph()
            
    def setDebug(self, debug):
        self.debug = debug
        
    def getNetwork(self):
        return self.g
    
    
    ################################################################################################
    # Vertices / Nodes / Location
    ################################################################################################    
    def showVertices(self):
        for nodename,node in self.g.nodes_iter(data=True):
            print(nodename,'\t',node)
                
    def showEdges(self):
        for edgename,edge in self.g.adj.items():
            print(edgename,'\t',edge)
                
                

        
    ################################################################################################
    # Vertices / Nodes / Location
    ################################################################################################    
    def addVertex(self, name, attrs={}):
        self.g.add_node(u=name, attr_dict=attrs)
        if self.debug:
            print("  Added node: [{0}]".format(", ".join(names)))
                    
    def updateVertexAttrs(self, attrs):
        if not isinstance(attrs, dict):
            print("Cannot add vertex attrs because the input is not a dict")
            return
        nx.set_node_attributes(G=self.g, values=attrs, name=None)
        self.orderVertices()
        
    def setNodeDict(self):
        ## There is some weirdness with the way the node attrs are initially stored
        self.nodeDict = {u: d for (u,d) in self.g.nodes(data=True)}
        #self.nodeDict = {k: v[None] for k,v in self.nodeDict.items()}
        
    def orderVertices(self, metric='Centrality'):
        self.setNodeDict()
        if metric == 'Centrality':
            from networkx.algorithms import degree_centrality
            tmp = degree_centrality(self.g)
        elif metric == 'Counts':
            tmp = {u: d['N'] for (u,d) in self.g.nodes(data=True)}
        else:
            raise ValueError("Metric {0} is not used for vertex ordering".format(metric))           
        self.orderedVertices = sorted(tmp, key=tmp.get, reverse=True)
        
    def getVertices(self, ordered=True):
        if self.orderedVertices is not None:
            return self.orderedVertices
        else:
            self.orderVertices()
            return self.orderedVertices
        
    def getVertexNumByName(self, name, debug=True):
        vertexList = self.getVertices()
        try:
            vertexNum = vertexList.index(name)
        except:
            if debug:
                print("Could not get vertex number for name {0}".format(name))
            vertexNum = None
        return vertexNum
        
    def getVertexData(self, vertexNum, debug=True):
        vertexList = self.getVertices()
        if vertexNum >= len(vertexList):
            if debug:
                print("Vertex num {0} is greater than Vertex list length {1}".format(vertexNum, len(vertexList)))
            return None
        try:
            vertexName = vertexList[vertexNum]
        except:
            if debug:
                print("Could not get Vertex name from Vertex list for num {0}".format(vertexNum))
            vertexName = None
        return self.getVertexDataByName(vertexName, debug=debug)
            
    def getVertexDataByName(self, name, debug=True):
        try:
            vertexData = self.nodeDict[name]
        except:
            if debug:
                print("Could not get Vertex data for Vertex name {0}".format(name))
            vertexData = None
        return vertexData        
            
        
        
    ################################################################################################
    # Edges / Trips
    ################################################################################################    
    def addEdge(self, names, attrs={}, sort=False):
        if not isinstance(names, (tuple,list,set)):
            print("Cannot add edge {0} because the names need to come in a tuple/list/set.".format(names))
            return
        if len(names) == 2:
            if sort is True:
                names = sorted([str(x) for x in names])
            else:
                names = [str(x) for x in names]
        else:
            print("Cannot add edge {0} because we need two entries in the tuple/list/set.".format(names))
            return
        
        self.g.add_edge(names[0], names[1], attr_dict=attrs)
        if self.debug:
            print("  Added edge: [{0}]".format(", ".join(names)))
            
    def updateEdgeAttrs(self, attrs):
        if not isinstance(attrs, dict):
            print("Cannot add edge attrs because the input is not a dict")
            return
        nx.set_edge_attributes(G=self.g, values=attrs)
        self.orderEdges()
        
    def setEdgeDict(self):
        self.edgeDict = {(u, v): d for (u,v,d) in self.g.edges(data=True)}
        
    def orderEdges(self, metric='Weight'):
        self.setEdgeDict()
        tmp = {(u,v): d[metric] for (u,v,d) in self.g.edges(data=True)}
        self.orderedEdges = sorted(tmp, key=tmp.get, reverse=True)
        
    def getEdges(self, ordered=True):
        if self.orderedEdges is not None:
            return self.orderedEdges
        else:
            self.orderEdges()
            return self.orderedEdges
        
    def getEdgeNumByName(self, name, debug=True):
        edgeList = self.getEdges()
        try:
            edgeNum = edgeList.index(name)
        except:
            if debug:
                print("Could not get edge number for name {0}".format(name))
            edgeNum = None
        return edgeNum
        
    def getEdgeData(self, edgeNum, debug=True):
        edgeList = self.getEdges()
        if edgeNum >= len(edgeList):
            if debug:
                print("Edge num {0} is greater than edge list length {1}".format(edgeNum, len(edgeList)))
            return None
        try:
            edgeName = edgeList[edgeNum]
        except:
            if debug:
                print("Could not get edge name from edge list for num {0}".format(edgeNum))
            edgeName = None
        return self.getEdgeDataByName(edgeName, debug=debug)
            
    def getEdgeDataByName(self, name, debug=True):
        try:
            edgeData = self.edgeDict[name]
        except:
            if debug:
                print("Could not get edge data for edge name {0}".format(name))
                print(self.edgeDict.keys())
            edgeData = None
        return edgeData

In [31]:
class driverNetwork(network):
    def __init__(self, trips):
        network.__init__(self, directed=False, debug=False)
        if trips is not None:
            if isinstance(trips, dict):
                self.name          = trips.get('device')
                self.edgeMetrics   = trips.get('edgeMetrics')
                self.vertexMetrics = trips.get('vertexMetrics')
                self.vertexMetrics = {str(k): v for k,v in self.vertexMetrics.items()}
                self.homeMetrics   = trips.get('homeMetrics')
                print("Creating a driver network with {0} vertices and {1} edges.".format(len(self.vertexMetrics), len(self.edgeMetrics)))
            else:
                raise ValueError("Input trips must be a dictionary of edgeMetrics, vertexMetrics, and homeMetrics (optional)")
        else:
            raise ValueError("Input trips is None!")

    def create(self):
        for edgename,edgedata in self.edgeMetrics.items():
            self.addEdge(edgename, edgedata)
        self.updateVertexAttrs(self.vertexMetrics)

In [32]:
dn = driverNetwork(trips)
dn.create()
g = dn.getNetwork()

Creating a driver network with 22 vertices and 207 edges.


In [141]:
#dn.nodeDict[dn.getVertices()[0]]
#dn.getVertexData(0)
#

# Load Data

In [17]:
#######################################################################################
# Generate Clusted Data
#######################################################################################
genData = True
if genData:
    genMax  = 75
    distMax = 500
    raw  = genClusters(20, 250, latRange=[29.8, 30.2], lngRange=[49.8, 50.2], dist="gauss", maxrad=genMax)
    gc   = geoClusters(key="dummy", points=raw, distMax=distMax, debug=False)
    gc.findClusters(seedMin=2, debug=False)
    df   = genTripsBetweenClusters(n=1000, gc=gc, returnDF=True)
else:
    df   = loadJoblib("trips.p")

Selected 1000 randomized trips
Found Start/End for the 1000 randomized trips
Converting (1000, 2, 2) trips to a DataFrame


In [18]:
# Show Data (if needed)
df.head()

,lat0,long0,lat1,long1,total_miles,duration,start,end
0,29.828202,49.923860,29.912829,50.191995,27.513606,550.272112,2018-11-07 13:21:15.650373,2018-11-07 13:30:25.922485
1,29.827215,49.923952,30.035841,49.912784,23.223164,464.463275,2018-11-07 13:21:15.650373,2018-11-07 13:29:00.113648
2,30.080868,49.819193,30.171888,49.820982,10.122464,202.449286,2018-11-07 13:21:15.650373,2018-11-07 13:24:38.099659
3,30.014167,49.918859,29.942636,50.171559,25.606321,512.126414,2018-11-07 13:21:15.650373,2018-11-07 13:29:47.776787
4,29.963853,50.019234,30.159408,50.013696,21.751160,435.023206,2018-11-07 13:21:15.650373,2018-11-07 13:28:30.673579


In [19]:
#######################################################################################
# Cluster Geo Data (Lat, Long)
#######################################################################################
points         = df[["lat0", "long0"]]
points.columns = ["lat", "long"]
pnts           = df[["lat1", "long1"]]
pnts.columns   = ["lat", "long"]    
points         = points.append(pnts)



#######################################################################################
# Create Clusters
#######################################################################################
debug=True
gc   = geoClusters(key="dummy", points=points, distMax=300, debug=debug)
gc.findClusters(seedMin=2, debug=debug)
if debug:
    print("Found {0} clusters using {1} cells and {2} counts".format(gc.getNClusters(), gc.getNCells(), gc.getNCounts()))

    
    
#######################################################################################
# Set Nearest Clusters
#######################################################################################
if debug:
    start, cmt = clock("Finding Nearest Clusters for Start of Trips")
geoResults = df[['lat0', 'long0']].apply(gc.getNearestClusters, axis=1).values
df["geo0"] = [x[0] for x in geoResults]
if debug:
    elapsed(start, cmt)
    start, cmt = clock("Finding Nearest Clusters for End of Trips")
geoResults = df[['lat1', 'long1']].apply(gc.getNearestClusters, axis=1).values
df["geo1"] = [x[0] for x in geoResults]    
if debug:
    elapsed(start, cmt)

    
    
#######################################################################################
# Organize Trips for Network
#######################################################################################
trips = organizeTrips(df=df, gc=gc, debug=False, requireGood=False)

Current Time is Wed Nov 07, 2018 13:21:18 for Converting 2000 Points To Correct Format
Data has correct format with a (2000, 2) shape.
Current Time is Wed Nov 07, 2018 13:21:18 for Done with Converting 2000 Points To Correct Format
Process [Done with Converting 2000 Points To Correct Format] took 0 seconds.
Current Time is Wed Nov 07, 2018 13:21:18 for Finding Geohash (BitLen=8) Values from 2000 Points
Current Time is Wed Nov 07, 2018 13:21:18 for Done with Finding Geohash (BitLen=8) Values from 2000 Points
Process [Done with Finding Geohash (BitLen=8) Values from 2000 Points] took 0 seconds.
Current Time is Wed Nov 07, 2018 13:21:18 for Finding Geohash (BitLen=8) Frequency Values from Geohash DataFrame
Current Time is Wed Nov 07, 2018 13:21:18 for Done with Finding Geohash (BitLen=8) Frequency Values from Geohash DataFrame
Process [Done with Finding Geohash (BitLen=8) Frequency Values from Geohash DataFrame] took 0 seconds.
Current Time is Wed Nov 07, 2018 13:21:18 for Finding Cluster

In [20]:
# Show data frame (if needed)
from pandasUtils import castDateTime
df['start'] = castDateTime(df['start'])
df['end'] = castDateTime(df['end'])
df.head()

,lat0,long0,lat1,long1,total_miles,duration,start,end,geo0,geo1
0,29.828202,49.923860,29.912829,50.191995,27.513606,550.272112,2018-11-07 13:21:15.650373,2018-11-07 13:30:25.922485,cl1,cl8
1,29.827215,49.923952,30.035841,49.912784,23.223164,464.463275,2018-11-07 13:21:15.650373,2018-11-07 13:29:00.113648,cl1,cl7
2,30.080868,49.819193,30.171888,49.820982,10.122464,202.449286,2018-11-07 13:21:15.650373,2018-11-07 13:24:38.099659,cl18,cl3
3,30.014167,49.918859,29.942636,50.171559,25.606321,512.126414,2018-11-07 13:21:15.650373,2018-11-07 13:29:47.776787,cl0,cl11
4,29.963853,50.019234,30.159408,50.013696,21.751160,435.023206,2018-11-07 13:21:15.650373,2018-11-07 13:28:30.673579,cl5,cl2


# Run the Network

In [1]:
from networkx import algorithms as algos

In [5]:
import networkx as nx
nx.__version__
nx.algorithms.approximation.steinertree.steiner_tree

AttributeError: module 'networkx.algorithms' has no attribute 'approximation'

In [93]:
def runAlgos(algosToRun, g):
    from types import GeneratorType
    results = {}
    for algo in algosToRun:
        name = algo.__name__

        try:
            results[name] = algo(g)
        except:
            results[name] = None

        if isinstance(results[name], GeneratorType):
            results[name] = len([1 for _ in results[name]])
            try:
                results[name] = len([1 for _ in algo])
            except:
                pass
                
                
    print(results)

In [77]:
algosToRun = []
algosToRun.append(algos.all_pairs_node_connectivity)
algosToRun.append(algos.node_connectivity)
algosToRun.append(algos.k_components)
algosToRun.append(algos.average_clustering)
#runAlgos(algosToRun, g)

In [78]:
algosToRun = []
algosToRun.append(algos.degree_assortativity_coefficient)
algosToRun.append(algos.degree_pearson_correlation_coefficient)
algosToRun.append(algos.average_neighbor_degree)
algosToRun.append(algos.average_degree_connectivity)
algosToRun.append(algos.k_nearest_neighbors)
algosToRun.append(algos.degree_mixing_matrix)
algosToRun.append(algos.degree_mixing_dict)
#runAlgos(algosToRun, g)

In [95]:
algosToRun = []
algosToRun.append(algos.is_bipartite)
algosToRun.append(algos.bipartite.sets)
#runAlgos(algosToRun, g)

{'is_bipartite': False, 'sets': None}


In [96]:
algosToRun = []
algosToRun.append(algos.bridges)
algosToRun.append(algos.has_bridges)
#runAlgos(algosToRun, g)

In [98]:
algosToRun = []
algosToRun.append(algos.degree_centrality)
algosToRun.append(algos.in_degree_centrality)
algosToRun.append(algos.out_degree_centrality)
algosToRun.append(algos.eigenvector_centrality)
algosToRun.append(algos.eigenvector_centrality_numpy)
algosToRun.append(algos.katz_centrality)
algosToRun.append(algos.katz_centrality_numpy)
algosToRun.append(algos.closeness_centrality)
algosToRun.append(algos.current_flow_closeness_centrality)
algosToRun.append(algos.information_centrality)
algosToRun.append(algos.betweenness_centrality)
algosToRun.append(algos.edge_betweenness_centrality)
algosToRun.append(algos.betweenness_centrality_subset)
algosToRun.append(algos.edge_betweenness_centrality_subset)
algosToRun.append(algos.current_flow_betweenness_centrality)
algosToRun.append(algos.edge_current_flow_betweenness_centrality)
algosToRun.append(algos.approximate_current_flow_betweenness_centrality)
algosToRun.append(algos.current_flow_betweenness_centrality_subset)
algosToRun.append(algos.edge_current_flow_betweenness_centrality_subset)
algosToRun.append(algos.communicability_betweenness_centrality)
algosToRun.append(algos.load_centrality)
algosToRun.append(algos.edge_load_centrality)
algosToRun.append(algos.subgraph_centrality)
algosToRun.append(algos.subgraph_centrality_exp)
algosToRun.append(algos.estrada_index)
algosToRun.append(algos.harmonic_centrality)
algosToRun.append(algos.local_reaching_centrality)
algosToRun.append(algos.global_reaching_centrality)
algosToRun.append(algos.percolation_centrality)
algosToRun.append(algos.second_order_centrality)
#runAlgos(algosToRun, g)

In [99]:
algosToRun = []
algosToRun.append(algos.chain_decomposition)
runAlgos(algosToRun, g)

{'chain_decomposition': 186}


In [104]:
algosToRun = []
algosToRun.append(algos.is_chordal)
algosToRun.append(algos.chordal_graph_cliques)
algosToRun.append(algos.chordal_graph_treewidth)
algosToRun.append(algos.find_induced_nodes)
#runAlgos(algosToRun, g)

In [103]:
algosToRun = []
algosToRun.append(algos.enumerate_all_cliques)
algosToRun.append(algos.find_cliques)
#algosToRun.append(algos.make_max_clique_graph)
#algosToRun.append(algos.make_clique_bipartite)
algosToRun.append(algos.graph_clique_number)
algosToRun.append(algos.graph_number_of_cliques)
algosToRun.append(algos.node_clique_number)
algosToRun.append(algos.number_of_cliques)
algosToRun.append(algos.cliques_containing_node)
#runAlgos(algosToRun, g)

In [106]:
algosToRun = []
algosToRun.append(algos.triangles)
algosToRun.append(algos.transitivity)
algosToRun.append(algos.clustering)
algosToRun.append(algos.average_clustering)
algosToRun.append(algos.square_clustering)
algosToRun.append(algos.generalized_degree)
#runAlgos(algosToRun, g)

In [108]:
algosToRun = []
algosToRun.append(algos.communicability)

In [109]:
algosToRun = []
algosToRun.append(algos.community.kernighan_lin_bisection)
algosToRun.append(algos.community.k_clique_communities)
algosToRun.append(algos.community.greedy_modularity_communities)
algosToRun.append(algos.community.asyn_lpa_communities)
algosToRun.append(algos.community.label_propagation_communities)
algosToRun.append(algos.community.asyn_fluidc)
algosToRun.append(algos.community.girvan_newman)
runAlgos(algosToRun, g)

{'kernighan_lin_bisection': ({'17', '1', '0', '10', '6', '11', '12', '7', '4', '20', '13'}, {'5', '16', '21', '8', '14', '15', '18', '3', '9', '19', '2'}), 'k_clique_communities': None, 'greedy_modularity_communities': [frozenset({'17', '0', '6', '3', '11', '12', '7', '4', '19', '20', '2'}), frozenset({'1', '21', '8', '10', '15', '18', '13'}), frozenset({'14', '16'}), frozenset({'5'}), frozenset({'9'})], 'asyn_lpa_communities': <dict_valueiterator object at 0x1a20bf9598>, 'label_propagation_communities': 1, 'asyn_fluidc': None, 'girvan_newman': 21}


In [110]:
algosToRun = []
algosToRun.append(algos.is_connected)
algosToRun.append(algos.number_connected_components)
algosToRun.append(algos.is_strongly_connected)
algosToRun.append(algos.number_strongly_connected_components)
algosToRun.append(algos.is_weakly_connected)
algosToRun.append(algos.number_weakly_connected_components)
algosToRun.append(algos.is_attracting_component)
algosToRun.append(algos.number_attracting_components)
algosToRun.append(algos.is_biconnected)
algosToRun.append(algos.articulation_points)
algosToRun.append(algos.is_semiconnected)
runAlgos(algosToRun, g)

{'is_connected': True, 'number_connected_components': 1, 'is_strongly_connected': None, 'number_strongly_connected_components': None, 'is_weakly_connected': None, 'number_weakly_connected_components': None, 'is_attracting_component': None, 'number_attracting_components': None, 'is_biconnected': True, 'articulation_points': 0, 'is_semiconnected': None}


In [36]:


if False:






    algosToRun = []
    algosToRun.append(algos.k_components)
    algosToRun.append(algos.all_node_cuts)
    algosToRun.append(algos.average_node_connectivity)
    algosToRun.append(algos.all_pairs_node_connectivity)
    algosToRun.append(algos.edge_connectivity)
    algosToRun.append(algos.node_connectivity)
    algosToRun.append(algos.minimum_edge_cut)
    algosToRun.append(algos.minimum_node_cut)
    algosToRun.append(algos.stoer_wagner)



In [190]:
nx.algorithms.connectivity.

AttributeError: module 'networkx.algorithms.connectivity' has no attribute 'edge_augmentation'

In [ ]:
x = {1: "hoi", 2: "test"}

In [ ]:
x

In [ ]:
x = {str(k): v for k,v in x.items()}

In [ ]:
x

In [23]:
help(nx.addEdge)

AttributeError: module 'networkx' has no attribute 'addEdge'